<a href="https://colab.research.google.com/github/susansarathomas95/Freezer-Analysis/blob/main/FreezerAnalysisFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
from collections import defaultdict
import numpy as np

# --- Load and Clean ---
df = pd.read_csv('/content/Freezer3.csv')
df['Temperature (celsius)'] = df['Temperature (celsius)'].fillna(method='ffill')
df['Door Contact '] = df['Door Contact '].fillna(method='ffill')
df.to_csv('Freezer3_filled.csv', index=False)
print("Missing values filled. Updated CSV saved as 'Freezer3_filled.csv'.")

# --- Reload and Sort ---
df = pd.read_csv('/content/Freezer3_filled.csv')
df['Date Time'] = pd.to_datetime(df['Date Time'], dayfirst=True)
df = df.sort_values('Date Time').reset_index(drop=True)

# --- User Inputs ---
normal_min = float(input("Enter minimum normal temperature (°C): "))
normal_max = float(input("Enter maximum normal temperature (°C): "))
spike_percent = float(input("Enter spike threshold percentage (e.g., 10 for 10%): "))
time_filter = input("Filter by 'month', 'date', 'range', or 'none': ").strip().lower()

normal_min, normal_max = min(normal_min, normal_max), max(normal_min, normal_max)
normal_range = normal_max - normal_min

# --- Apply Filters ---
if time_filter == 'month':
    month = int(input("Enter month (1-12): "))
    df = df[df['Date Time'].dt.month == month].reset_index(drop=True)
elif time_filter == 'date':
    date_str = input("Enter date (DD-MM-YYYY): ").strip()
    target_date = pd.to_datetime(date_str, format='%d-%m-%Y').date()
    df = df[df['Date Time'].dt.date == target_date].reset_index(drop=True)
elif time_filter == 'range':
    start_date = pd.to_datetime(input("Enter start date (DD-MM-YYYY): "), format='%d-%m-%Y')
    end_date = pd.to_datetime(input("Enter end date (DD-MM-YYYY): "), format='%d-%m-%Y')
    df = df[(df['Date Time'] >= start_date) & (df['Date Time'] <= end_date)].reset_index(drop=True)
elif time_filter != 'none':
    print("Invalid filter option.")
    exit()

if df.empty:
    print("No data after filtering. Exiting.")
    exit()

# --- Thresholds ---
spike_thresh_upper = normal_max + ((spike_percent / 100) * normal_range)
spike_thresh_lower = normal_min - ((spike_percent / 100) * normal_range)
print(f"Spike Upper Threshold: {spike_thresh_upper:.2f}°C")
print(f"Spike Lower Threshold: {spike_thresh_lower:.2f}°C")

# --- Spike Detection ---
def detect_spikes(df, upper, lower):
    events, in_spike, start_idx = [], False, None
    for i in range(1, len(df)):
        t, pt = df.loc[i, 'Temperature (celsius)'], df.loc[i - 1, 'Temperature (celsius)']
        if not in_spike and ((pt <= upper and t > upper) or (pt >= lower and t < lower)):
            in_spike, start_idx = True, i
        elif in_spike and (lower <= t <= upper):
            events.append((start_idx, i))
            in_spike = False
    if in_spike and start_idx is not None:
        events.append((start_idx, len(df) - 1))
    return events

# --- Helper: Door duration calculation ---
def summarize_door_states(door_series, time_series):
    durations = {'Open': 0.0, 'Close': 0.0}
    current_state = door_series.iloc[0]
    start_time = time_series.iloc[0]
    for i in range(1, len(door_series)):
        if door_series.iloc[i] != current_state:
            end_time = time_series.iloc[i]
            duration = (end_time - start_time).total_seconds() / 60
            if current_state in durations:
                durations[current_state] += duration
            current_state = door_series.iloc[i]
            start_time = time_series.iloc[i]
    end_time = time_series.iloc[-1]
    duration = (end_time - start_time).total_seconds() / 60
    if current_state in durations:
        durations[current_state] += duration
    return {k: round(v, 2) for k, v in durations.items()}

# --- Gap, Correlation, Transitions ---
def analyze_spike_patterns(summary):
    gaps = [(summary[i]['Start Time'] - summary[i - 1]['End Time']).total_seconds() / 60 for i in range(1, len(summary))]
    return {
        'Average Gap (min)': round(np.mean(gaps), 2) if gaps else 0,
        'Min Gap (min)': round(np.min(gaps), 2) if gaps else 0,
        'Max Gap (min)': round(np.max(gaps), 2) if gaps else 0,
        'Std Deviation (min)': round(np.std(gaps), 2) if gaps else 0
    }

def analyze_hourly_door_correlation(summary):
    combo = defaultdict(int)
    hour_count = defaultdict(int)
    for s in summary:
        hour = s['Start Time'].hour
        door = s['Door Status at Start']
        combo[(hour, door)] += 1
        hour_count[hour] += 1
    if not hour_count:
        return "No hourly spike patterns found."
    top_hour = max(hour_count, key=hour_count.get)
    dom_state = max([(k, v) for k, v in combo.items() if k[0] == top_hour], key=lambda x: x[1])[0][1]
    return f"Most spikes ({hour_count[top_hour]}) occur around {top_hour}:00, primarily when the door is {dom_state}."

def analyze_door_transition_patterns(summary_df):
    no_transitions = summary_df[summary_df['Door Transitions During Spike'] == 0].shape[0]
    one_transition = summary_df[summary_df['Door Transitions During Spike'] == 1].shape[0]
    multiple_transitions = summary_df[summary_df['Door Transitions During Spike'] > 1].shape[0]
    total = len(summary_df)
    if total == 0:
        return "No spike data to analyze door transitions."
    insights = []
    if no_transitions > 0:
        insights.append(f"{no_transitions} spike(s) showed no door transitions")
    if one_transition > 0:
        insights.append(f"{one_transition} spike(s) involved exactly one door event")
    if multiple_transitions > 0:
        insights.append(f"{multiple_transitions} spike(s) had multiple door transitions")
    no_transition_pct = (no_transitions / total) * 100
    one_transition_pct = (one_transition / total) * 100
    multiple_transition_pct = (multiple_transitions / total) * 100
    if no_transition_pct > max(one_transition_pct, multiple_transition_pct):
        conclusion = ("Majority of spikes occurred without any door transitions, "
                      "suggesting these spikes might be due to internal or systemic causes.")
    elif multiple_transition_pct > max(no_transition_pct, one_transition_pct):
        conclusion = ("Most spikes coincide with multiple door transitions, indicating "
                      "likely caused by frequent door usage.")
    elif one_transition_pct > max(no_transition_pct, multiple_transition_pct):
        conclusion = ("Many spikes correspond to exactly one door transition, possibly indicating "
                      "brief door openings.")
    else:
        conclusion = ("Spike causes are mixed between door activity and other factors.")
    insights.append("")  # blank line before conclusion
    insights.append("Conclusion: " + conclusion)
    return "\n".join(insights)

# --- Main Analysis ---
spike_events = detect_spikes(df, spike_thresh_upper, spike_thresh_lower)
spike_summary = []
door_duration_per_spike = []
for i, (start, end) in enumerate(spike_events, 1):
    st, et = df.loc[start, 'Date Time'], df.loc[end, 'Date Time']
    spike_df = df.iloc[start:end + 1]
    door_series = spike_df['Door Contact '].reset_index(drop=True)
    door_transitions = sum(door_series[i] != door_series[i - 1] for i in range(1, len(door_series)))
    door_durations = summarize_door_states(spike_df['Door Contact '], spike_df['Date Time'])

    spike_summary.append({
        'Spike Number': i,
        'Start Time': st,
        'End Time': et,
        'Duration (min)': round((et - st).total_seconds() / 60, 2),
        'Start Temp (°C)': round(df.loc[start, 'Temperature (celsius)'], 2),
        'End Temp (°C)': round(df.loc[end, 'Temperature (celsius)'], 2),
        'Door Status at Start': df.loc[start, 'Door Contact '],
        'Door Status at End': df.loc[end, 'Door Contact '],
        'Door Transitions During Spike': door_transitions,
        'Door Open Duration (min)': door_durations.get('Open', 0.0),
        'Door Close Duration (min)': door_durations.get('Close', 0.0)
    })

df_spike_summary = pd.DataFrame(spike_summary)

# --- Door Status at Spike Start ---
start_status_counts = df_spike_summary['Door Status at Start'].value_counts().to_dict()
door_open_at_start = start_status_counts.get('Open', 0)
door_close_at_start = start_status_counts.get('Close', 0)

# --- Gap and Insight Analysis ---
gap_stats = analyze_spike_patterns(spike_summary)
spike_frequency = f"Spikes occur approximately every {gap_stats['Average Gap (min)']} minutes." if gap_stats['Average Gap (min)'] else "Insufficient spike data."
hour_door_insight = analyze_hourly_door_correlation(spike_summary)
door_transition_insights = analyze_door_transition_patterns(df_spike_summary)

# --- Average Spikes ---
if not df_spike_summary.empty:
    total_duration = (df_spike_summary['End Time'].max() - df_spike_summary['Start Time'].min()).total_seconds()
    duration_hours = total_duration / 3600
    duration_days = total_duration / (3600 * 24)
    duration_months = duration_days / 30.44
    avg_hourly = round(len(df_spike_summary) / duration_hours, 2) if duration_hours else 0
    avg_daily = round(len(df_spike_summary) / duration_days, 2) if duration_days else 0
    avg_monthly = round(len(df_spike_summary) / duration_months, 2) if duration_months else 0
    avg_spike_summary = f"On average, there are {avg_hourly} spikes per hour, {avg_daily} spikes per day, {avg_monthly} spikes per month."

# --- Summary Output ---
summary_lines = [
    f"Normal Temperature Range: {normal_min:.2f} - {normal_max:.2f} °C",
    f"Spike Threshold: ±{spike_percent}%",
    f"Spike Upper Threshold: {spike_thresh_upper:.2f} °C",
    f"Spike Lower Threshold: {spike_thresh_lower:.2f} °C",
    f"Total Spikes Detected: {len(spike_events)}",
    "",
    avg_spike_summary,
    "",
    f"Spikes Started with Door 'Open': {door_open_at_start}",
    f"Spikes Started with Door 'Close': {door_close_at_start}",
    "",
    "Spike Gaps:",
    f"Average Gap: {gap_stats['Average Gap (min)']} min",
    f"Min Gap: {gap_stats['Min Gap (min)']} min",
    f"Max Gap: {gap_stats['Max Gap (min)']} min",
    f"Std Deviation: {gap_stats['Std Deviation (min)']} min",
    spike_frequency,
    "",
    "Spike Timing Insight:",
    hour_door_insight,
    "",
    "Door Transition Insights:"
]
summary_lines.extend(door_transition_insights.split('\n'))

# --- Export to Excel ---
output_path = '/content/freezer3_analysis_results.xlsx'
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    pd.DataFrame({'Summary': summary_lines}).to_excel(writer, sheet_name='Console Summary', index=False)
    df_spike_summary.to_excel(writer, sheet_name='Spike Summary', index=False)

# --- Console Output ---
print("\n--- ANALYSIS SUMMARY ---")
for line in summary_lines:
    print(line)
print(f"\nResults saved to: {output_path}")


/tmp/ipython-input-1-596558975.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Temperature (celsius)'] = df['Temperature (celsius)'].fillna(method='ffill')
/tmp/ipython-input-1-596558975.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Door Contact '] = df['Door Contact '].fillna(method='ffill')


Missing values filled. Updated CSV saved as 'Freezer3_filled.csv'.
Enter minimum normal temperature (°C): -5
Enter maximum normal temperature (°C): -9
Enter spike threshold percentage (e.g., 10 for 10%): 20
Filter by 'month', 'date', 'range', or 'none': none
Spike Upper Threshold: -4.20°C
Spike Lower Threshold: -9.80°C

--- ANALYSIS SUMMARY ---
Normal Temperature Range: -9.00 - -5.00 °C
Spike Threshold: ±20.0%
Spike Upper Threshold: -4.20 °C
Spike Lower Threshold: -9.80 °C
Total Spikes Detected: 91

On average, there are 0.19 spikes per hour, 4.61 spikes per day, 140.39 spikes per month.

Spikes Started with Door 'Open': 36
Spikes Started with Door 'Close': 55

Spike Gaps:
Average Gap: 21.09 min
Min Gap: 10.0 min
Max Gap: 70.0 min
Std Deviation: 16.61 min
Spikes occur approximately every 21.09 minutes.

Spike Timing Insight:
Most spikes (12) occur around 23:00, primarily when the door is Open.

Door Transition Insights:
15 spike(s) showed no door transitions
10 spike(s) involved exactl